# Deep Learning - Gesture Recognition Case Study #

### Problem Statement
Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:
 
| Gesture | Corresponding Action |
| --- | --- | 
| Thumbs Up | Increase the volume. |
| Thumbs Down | Decrease the volume. |
| Left Swipe | 'Jump' backwards 10 seconds. |
| Right Swipe | 'Jump' forward 10 seconds. |
| Stop | Pause the movie. |

Each video is a sequence of 30 frames (or images).

### Objectives:
1. **Generator**:  The generator should be able to take a batch of videos as input without any error. Steps like cropping, resizing and normalization should be performed successfully.

2. **Model**: Develop a model that is able to train without any errors which will be judged on the total number of parameters (as the inference(prediction) time should be less) and the accuracy achieved. As suggested by Snehansu, start training on a small amount of data and then proceed further.

3. **Write up**: This should contain the detailed procedure followed in choosing the final model. The write up should start with the reason for choosing the base model, then highlight the reasons and metrics taken into consideration to modify and experiment to arrive at the final model.

In [3]:
#Install Few Dependencies if not available and validated versions

#!pip install Pillow
#!pip install tensorflow
#!pip install keras
print("*"*30)
print("Versions")
print("*"*30)
!pip freeze | grep 'scipy\|keras\|tensorflow'
!nvidia-smi

******************************
Versions
******************************
horovod @ file:///opt/tensorflow/horovod-source/dist/horovod-0.20.2.tar.gz
scipy==1.4.1
tensorflow @ file:///tmp/pip/tensorflow-2.3.1%2Bnv-cp36-cp36m-linux_x86_64.whl
tensorflow-addons @ file:///opt/tensorflow/tf-addons/artifacts/tensorflow_addons-0.11.2-cp36-cp36m-linux_x86_64.whl
tensorflow-datasets==3.2.1
tensorflow-estimator==2.3.0
tensorflow-metadata==0.25.0
/bin/bash: nvidia-smi: command not found


In [6]:
#Import the necessary Libraries

import numpy as np
import os
from imageio import imread
import scipy
#imgresize is deprecated, use Pillow instead
#numpy.array(Image.fromarray(arr).resize())
#https://docs.scipy.org/doc/scipy-1.2.0/reference/generated/scipy.misc.imresize.html
from skimage.transform import resize
import datetime
import os
from tensorflow import keras
import tensorflow as tf


In [9]:
#Initialize Paths
import os
#help(os)
print(os.getcwd())
project_folder = '/datasets/Project_data'
train_image_metadata=project_folder+'/train.csv'
val_image_metadata=project_folder+'/val.csv'

/home/dl_content/Upgrad DL/Gesture Recognition Assignment/Gesture-Recognition-CaseStudy-IITB


We set the random seed so that the results don't vary drastically.

In [10]:
np.random.seed(30)
import random as rn
rn.seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [11]:
#train_doc = np.random.permutation(open(train_image_metadata).readlines())
#val_doc = np.random.permutation(open(val_image_metadata).readlines())
#batch_size = #experiment with the batch size

### Results Plotter ###

In [12]:
#Use the history object to plot the results

def plot(history):
    fig,axes = plt.subplots(nrows=1,ncols=2, figsize=(15,4))
    axes[0].plot(history.history['loss'])
    axes[0].plot(history.history['val_loss'])
    axes[0].legend(['loss','val_loss'])
    
    axes[1].plot(history.history['categorical_accuracy'])   
    axes[1].plot(history.history['val_categorical_accuracy'])
    axes[1].legend(['categorical_accuracy','val_categorical_accuracy'])

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [41]:
#Base Class for all the classes with common functionality
import abc
class BaseModel(metaclass=abc.ABCMeta):
    
    
    
    IMAGE_TRAIN_PATH=project_folder+'train'
    IMAGE_VAL_PATH=project_folder+'val'
    
    #initialize the path for the train and validation images
    #This will be called by each inherited class
    def initialize_path(self, project_folder):
        self.train_doc = np.random.permutation(open(train_image_metadata).readlines())
        self.val_doc = np.random.permutation(open(val_image_metadata).readlines())
        self.train_path = project_folder + '/' + 'train'
        self.val_path =  project_folder + '/' + 'val'
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
        
    
    #initialize the properties of each image
    #We will standardize all image using the 
    def initialize_image_properties(self,image_height=100,image_width=100):
        self.image_height=image_height
        self.image_width=image_width
        self.channels=3
        self.num_classes=5
        self.total_frames=30
    
    #initalize the training parameters, this will be used by  each model to train 
    def initialize_hyperparams(self,frames_to_sample=30,batch_size=20,num_epochs=20):
        self.frames_to_sample=frames_to_sample
        self.batch_size=batch_size
        self.num_epochs=num_epochs
        
    def generator(self, source_path, folder_list, augment=False):
        batch_size = self.batch_size
        print('Source path = ', source_path)
        print('batch size =', batch_size)
        img_idx = np.round(np.linspace(0,self.total_frames-1,self.frames_to_sample)).astype(int)
        while True:
            t = np.random.permutation(folder_list)
            print("t : ", t)
            num_batches =  len(t)//batch_size
            print("num_batches", num_batches)
            
            for batch in range(num_batches): # we iterate over the number of batches
                batch_data, batch_labels= self.one_batch_data_generator(source_path,t,batch,batch_size,img_idx,augment)
                yield batch_data, batch_labels 
                
            
            remaining_seq=len(t)%batch_size
        
            if (remaining_seq != 0):
                batch_data, batch_labels= self.one_batch_data_generator(source_path,t,num_batches,batch_size,img_idx,augment,remaining_seq)
                yield batch_data, batch_labels 
                
                
    # write the code for the remaining data points which are left after full batches
    def one_batch_data_generator(self, source_path, t, batch, batch_size, img_idx, augment, remaining_seq=0):
        
        #Initialize the seq length
        seq_len = remaining_seq if remaining_seq else batch_size
        batch_data = np.zeros((seq_len,len(img_idx),self.image_height,self.image_width,self.channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((seq_len,self.num_classes)) # batch_labels is the one hot representation of the output

        for folder in range(seq_len): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    print(idx)
                    print(item)
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    print(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image_resized=resize(image,(self.image_height,self.image_width,self.channels))
                    print(image_resized.shape)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255

                    if (augment):
                    #TO DO - Write the augmentation logic using OpenCV here
                        print("Augmentation to be written")
                    


                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

                #Add to the augmentated labels also if augmented is true


        return (batch_data, batch_labels)
    
    def train_model(self, model, augment_data = False):
        train_generator = self.generator(self.train_path, self.train_doc, augment= augment_data)
        val_generator = self.generator(self.val_path, self.val_doc)
        
        model_name = 'model_init' + '_' + str(datetime.datetime.now()).replace(' ','').replace(':','_') + '/'
        
        if not os.path.exists(model_name):
            os.mkdir(model_name)
            
        filepath = 'modle_init' + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}--{val_categorical_accuracy:.5f}.h5'
        
        #HyperParameter Tuners
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq = self.num_epochs)
        LR = ReduceLROnPlateau(monitor='val_loss', min_dela=0, patience=10, verbose=1)
        earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
        
        callbacks_list = [checkpoint, LR, earlystop]
        
        if (self.num_train_sequences%self.batch_size) == 0:
            steps_per_epoch = int(self.num_train_sequences/self.batch_size)
        else:
            steps_per_epoch = (self.num_train_sequences//self.batch_size) + 1

        if (self.num_val_sequences%self.batch_size) == 0:
            validation_steps = int(self.num_val_sequences/self.batch_size)
        else:
            validation_steps = (self.num_val_sequences//self.batch_size) + 1
            
        history=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=self.num_epochs, verbose=1, 
                            callbacks=callbacks_list, validation_data=val_generator, 
                            validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
        return history   
    
    @abc.abstractmethod
    def define_model(self):
        pass   

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Modelling Iterations
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [42]:
#Common Import for Models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers



### Initial Model : 1

In [43]:
class ModelConv3D_Inital(BaseModel):
    
    def define_model(self):
        model = Sequential()
        model.add(Conv3D(16, (3,3,3), padding='same',
                        input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2,2,2)))
        
        model.add(Conv3D(32,(2,2,2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2,2,2)))
        
        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(128,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(64,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(self.num_classes,activation='softmax'))
        
        optimizer = optimizers.Adam()
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

Assembling Model 1:

In [44]:
conv3d_initial = ModelConv3D_Inital()
conv3d_initial.initialize_path(project_folder)
conv3d_initial.initialize_image_properties(image_height=160, image_width=160)
conv3d_initial.initialize_hyperparams(frames_to_sample=30,batch_size=40,num_epochs=1)
conv3d_initial_model = conv3d_initial.define_model()
conv3d_initial_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_24 (Conv3D)           (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_24 (Activation)   (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_36 (Batc (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_24 (MaxPooling (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_25 (Activation)   (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_37 (Batc (None, 15, 80, 80, 32)   

In [ ]:
conv3d_initial.train_model(conv3d_initial_model)

Source path =  /datasets/Project_data/train
batch size = 40
t :  ['WIN_20180907_15_47_19_Pro_Thumbs Up_new;Thumbs Up_new;4\n'
 'WIN_20180926_17_33_51_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180926_17_19_35_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180926_16_50_23_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180907_16_25_52_Pro_Thumbs Down_new;Thumbs Down_new;3\n'
 'WIN_20180907_16_25_47_Pro_Left Swipe_new_Left Swipe_new;Left Swipe_new_Left Swipe_new;0\n'
 'WIN_20180926_17_37_02_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180926_16_48_34_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180907_16_15_15_Pro_Thumbs Down_new;Thumbs Down_new;3\n'
 'WIN_20180925_17_19_51_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180907_16_02_07_Pro_Stop Gesture_new;Stop Gesture_new;2\n'
 'WIN_20180907_15_43_40_Pro_Stop Gesture_new;Stop Gesture_new;2\n'
 'WIN_20180926_17_15_18_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180926_18_05_38_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180907_15_56_26_Pro_Thumbs Up_new;Thumbs Up

/datasets/Project_data/train/WIN_20180907_15_47_19_Pro_Thumbs Up_new/WIN_20180907_15_47_19_Pro_00037.png
(160, 160, 3)
17
17
/datasets/Project_data/train/WIN_20180907_15_47_19_Pro_Thumbs Up_new/WIN_20180907_15_47_19_Pro_00039.png
(160, 160, 3)
18
18
/datasets/Project_data/train/WIN_20180907_15_47_19_Pro_Thumbs Up_new/WIN_20180907_15_47_19_Pro_00041.png
(160, 160, 3)
19
19
/datasets/Project_data/train/WIN_20180907_15_47_19_Pro_Thumbs Up_new/WIN_20180907_15_47_19_Pro_00043.png
(160, 160, 3)
20
20
/datasets/Project_data/train/WIN_20180907_15_47_19_Pro_Thumbs Up_new/WIN_20180907_15_47_19_Pro_00045.png
(160, 160, 3)
21
21
/datasets/Project_data/train/WIN_20180907_15_47_19_Pro_Thumbs Up_new/WIN_20180907_15_47_19_Pro_00047.png
(160, 160, 3)
22
22
/datasets/Project_data/train/WIN_20180907_15_47_19_Pro_Thumbs Up_new/WIN_20180907_15_47_19_Pro_00049.png
(160, 160, 3)
23
23
/datasets/Project_data/train/WIN_20180907_15_47_19_Pro_Thumbs Up_new/WIN_20180907_15_47_19_Pro_00051.png
(160, 160, 3)
24
24


(160, 160, 3)
2
2
/datasets/Project_data/train/WIN_20180907_16_25_52_Pro_Thumbs Down_new/WIN_20180907_16_25_52_Pro_00007.png
(160, 160, 3)
3
3
/datasets/Project_data/train/WIN_20180907_16_25_52_Pro_Thumbs Down_new/WIN_20180907_16_25_52_Pro_00008.png
(160, 160, 3)
4
4
/datasets/Project_data/train/WIN_20180907_16_25_52_Pro_Thumbs Down_new/WIN_20180907_16_25_52_Pro_00009.png
(160, 160, 3)
5
5
/datasets/Project_data/train/WIN_20180907_16_25_52_Pro_Thumbs Down_new/WIN_20180907_16_25_52_Pro_00010.png
(160, 160, 3)
6
6
/datasets/Project_data/train/WIN_20180907_16_25_52_Pro_Thumbs Down_new/WIN_20180907_16_25_52_Pro_00011.png
(160, 160, 3)
7
7
/datasets/Project_data/train/WIN_20180907_16_25_52_Pro_Thumbs Down_new/WIN_20180907_16_25_52_Pro_00012.png
(160, 160, 3)
8
8
/datasets/Project_data/train/WIN_20180907_16_25_52_Pro_Thumbs Down_new/WIN_20180907_16_25_52_Pro_00013.png
(160, 160, 3)
9
9
/datasets/Project_data/train/WIN_20180907_16_25_52_Pro_Thumbs Down_new/WIN_20180907_16_25_52_Pro_00014.png


(160, 160, 3)
25
25
/datasets/Project_data/train/WIN_20180926_17_37_02_Pro_Stop_new/WIN_20180926_17_37_02_Pro_00051.png
(160, 160, 3)
26
26
/datasets/Project_data/train/WIN_20180926_17_37_02_Pro_Stop_new/WIN_20180926_17_37_02_Pro_00053.png
(160, 160, 3)
27
27
/datasets/Project_data/train/WIN_20180926_17_37_02_Pro_Stop_new/WIN_20180926_17_37_02_Pro_00055.png
(160, 160, 3)
28
28
/datasets/Project_data/train/WIN_20180926_17_37_02_Pro_Stop_new/WIN_20180926_17_37_02_Pro_00057.png
(160, 160, 3)
29
29
/datasets/Project_data/train/WIN_20180926_17_37_02_Pro_Stop_new/WIN_20180926_17_37_02_Pro_00059.png
(160, 160, 3)
0
0
/datasets/Project_data/train/WIN_20180926_16_48_34_Pro_Left_Swipe_new/WIN_20180926_16_48_34_Pro_00006.png
(160, 160, 3)
1
1
/datasets/Project_data/train/WIN_20180926_16_48_34_Pro_Left_Swipe_new/WIN_20180926_16_48_34_Pro_00008.png
(160, 160, 3)
2
2
/datasets/Project_data/train/WIN_20180926_16_48_34_Pro_Left_Swipe_new/WIN_20180926_16_48_34_Pro_00010.png
(160, 160, 3)
3
3
/datasets/

(160, 160, 3)
16
16
/datasets/Project_data/train/WIN_20180925_17_19_51_Pro_Left_Swipe_new/WIN_20180925_17_19_51_Pro_00019.png
(160, 160, 3)
17
17
/datasets/Project_data/train/WIN_20180925_17_19_51_Pro_Left_Swipe_new/WIN_20180925_17_19_51_Pro_00020.png
(160, 160, 3)
18
18
/datasets/Project_data/train/WIN_20180925_17_19_51_Pro_Left_Swipe_new/WIN_20180925_17_19_51_Pro_00021.png
(160, 160, 3)
19
19
/datasets/Project_data/train/WIN_20180925_17_19_51_Pro_Left_Swipe_new/WIN_20180925_17_19_51_Pro_00022.png
(160, 160, 3)
20
20
/datasets/Project_data/train/WIN_20180925_17_19_51_Pro_Left_Swipe_new/WIN_20180925_17_19_51_Pro_00023.png
(160, 160, 3)
21
21
/datasets/Project_data/train/WIN_20180925_17_19_51_Pro_Left_Swipe_new/WIN_20180925_17_19_51_Pro_00024.png
(160, 160, 3)
22
22
/datasets/Project_data/train/WIN_20180925_17_19_51_Pro_Left_Swipe_new/WIN_20180925_17_19_51_Pro_00025.png
(160, 160, 3)
23
23
/datasets/Project_data/train/WIN_20180925_17_19_51_Pro_Left_Swipe_new/WIN_20180925_17_19_51_Pro_00

...

/datasets/Project_data/train/WIN_20180926_17_28_52_Pro_Right_Swipe_new/WIN_20180926_17_28_52_Pro_00041.png
(160, 160, 3)
19
19
/datasets/Project_data/train/WIN_20180926_17_28_52_Pro_Right_Swipe_new/WIN_20180926_17_28_52_Pro_00043.png
(160, 160, 3)
20
20
/datasets/Project_data/train/WIN_20180926_17_28_52_Pro_Right_Swipe_new/WIN_20180926_17_28_52_Pro_00045.png
(160, 160, 3)
21
21
/datasets/Project_data/train/WIN_20180926_17_28_52_Pro_Right_Swipe_new/WIN_20180926_17_28_52_Pro_00047.png
(160, 160, 3)
22
22
/datasets/Project_data/train/WIN_20180926_17_28_52_Pro_Right_Swipe_new/WIN_20180926_17_28_52_Pro_00049.png
(160, 160, 3)
23
23
/datasets/Project_data/train/WIN_20180926_17_28_52_Pro_Right_Swipe_new/WIN_20180926_17_28_52_Pro_00051.png
(160, 160, 3)
24
24
/datasets/Project_data/train/WIN_20180926_17_28_52_Pro_Right_Swipe_new/WIN_20180926_17_28_52_Pro_00053.png
(160, 160, 3)
25
25
/datasets/Project_data/train/WIN_20180926_17_28_52_Pro_Right_Swipe_new/WIN_20180926_17_28_52_Pro_00055.png
(160

(160, 160, 3)
7
7
/datasets/Project_data/train/WIN_20180926_17_51_06_Pro_Stop_new/WIN_20180926_17_51_06_Pro_00019.png
(160, 160, 3)
8
8
/datasets/Project_data/train/WIN_20180926_17_51_06_Pro_Stop_new/WIN_20180926_17_51_06_Pro_00021.png
(160, 160, 3)
9
9
/datasets/Project_data/train/WIN_20180926_17_51_06_Pro_Stop_new/WIN_20180926_17_51_06_Pro_00023.png
(160, 160, 3)
10
10
/datasets/Project_data/train/WIN_20180926_17_51_06_Pro_Stop_new/WIN_20180926_17_51_06_Pro_00025.png
(160, 160, 3)
11
11
/datasets/Project_data/train/WIN_20180926_17_51_06_Pro_Stop_new/WIN_20180926_17_51_06_Pro_00027.png
(160, 160, 3)
12
12
/datasets/Project_data/train/WIN_20180926_17_51_06_Pro_Stop_new/WIN_20180926_17_51_06_Pro_00029.png
(160, 160, 3)
13
13
/datasets/Project_data/train/WIN_20180926_17_51_06_Pro_Stop_new/WIN_20180926_17_51_06_Pro_00031.png
(160, 160, 3)
14
14
/datasets/Project_data/train/WIN_20180926_17_51_06_Pro_Stop_new/WIN_20180926_17_51_06_Pro_00033.png
(160, 160, 3)
15
15
/datasets/Project_data/tra

(160, 160, 3)
0
0
/datasets/Project_data/train/WIN_20180926_17_21_38_Pro_Thumbs_Up_new/WIN_20180926_17_21_38_Pro_00006.png
(160, 160, 3)
1
1
/datasets/Project_data/train/WIN_20180926_17_21_38_Pro_Thumbs_Up_new/WIN_20180926_17_21_38_Pro_00008.png
(160, 160, 3)
2
2
/datasets/Project_data/train/WIN_20180926_17_21_38_Pro_Thumbs_Up_new/WIN_20180926_17_21_38_Pro_00010.png
(160, 160, 3)
3
3
/datasets/Project_data/train/WIN_20180926_17_21_38_Pro_Thumbs_Up_new/WIN_20180926_17_21_38_Pro_00012.png
(160, 160, 3)
4
4
/datasets/Project_data/train/WIN_20180926_17_21_38_Pro_Thumbs_Up_new/WIN_20180926_17_21_38_Pro_00014.png
(160, 160, 3)
5
5
/datasets/Project_data/train/WIN_20180926_17_21_38_Pro_Thumbs_Up_new/WIN_20180926_17_21_38_Pro_00016.png
(160, 160, 3)
6
6
/datasets/Project_data/train/WIN_20180926_17_21_38_Pro_Thumbs_Up_new/WIN_20180926_17_21_38_Pro_00018.png
(160, 160, 3)
7
7
/datasets/Project_data/train/WIN_20180926_17_21_38_Pro_Thumbs_Up_new/WIN_20180926_17_21_38_Pro_00020.png
(160, 160, 3)
8


(160, 160, 3)
9
9
/datasets/Project_data/train/WIN_20180907_16_01_35_Pro_Thumbs Down_new/WIN_20180907_16_01_35_Pro_00019.png
(160, 160, 3)
10
10
/datasets/Project_data/train/WIN_20180907_16_01_35_Pro_Thumbs Down_new/WIN_20180907_16_01_35_Pro_00020.png
(160, 160, 3)
11
11
/datasets/Project_data/train/WIN_20180907_16_01_35_Pro_Thumbs Down_new/WIN_20180907_16_01_35_Pro_00021.png
(160, 160, 3)
12
12
/datasets/Project_data/train/WIN_20180907_16_01_35_Pro_Thumbs Down_new/WIN_20180907_16_01_35_Pro_00022.png
(160, 160, 3)
13
13
/datasets/Project_data/train/WIN_20180907_16_01_35_Pro_Thumbs Down_new/WIN_20180907_16_01_35_Pro_00023.png
(160, 160, 3)
14
14
/datasets/Project_data/train/WIN_20180907_16_01_35_Pro_Thumbs Down_new/WIN_20180907_16_01_35_Pro_00024.png
(160, 160, 3)
15
15
/datasets/Project_data/train/WIN_20180907_16_01_35_Pro_Thumbs Down_new/WIN_20180907_16_01_35_Pro_00025.png
(160, 160, 3)
16
16
/datasets/Project_data/train/WIN_20180907_16_01_35_Pro_Thumbs Down_new/WIN_20180907_16_01_35_

(160, 160, 3)
19
19
/datasets/Project_data/train/WIN_20180907_16_13_12_Pro_Stop Gesture_new/WIN_20180907_16_13_12_Pro_00052.png
(160, 160, 3)
20
20
/datasets/Project_data/train/WIN_20180907_16_13_12_Pro_Stop Gesture_new/WIN_20180907_16_13_12_Pro_00054.png
(160, 160, 3)
21
21
/datasets/Project_data/train/WIN_20180907_16_13_12_Pro_Stop Gesture_new/WIN_20180907_16_13_12_Pro_00056.png
(160, 160, 3)
22
22
/datasets/Project_data/train/WIN_20180907_16_13_12_Pro_Stop Gesture_new/WIN_20180907_16_13_12_Pro_00058.png
(160, 160, 3)
23
23
/datasets/Project_data/train/WIN_20180907_16_13_12_Pro_Stop Gesture_new/WIN_20180907_16_13_12_Pro_00060.png
(160, 160, 3)
24
24
/datasets/Project_data/train/WIN_20180907_16_13_12_Pro_Stop Gesture_new/WIN_20180907_16_13_12_Pro_00062.png
(160, 160, 3)
25
25
/datasets/Project_data/train/WIN_20180907_16_13_12_Pro_Stop Gesture_new/WIN_20180907_16_13_12_Pro_00064.png
(160, 160, 3)
26
26
/datasets/Project_data/train/WIN_20180907_16_13_12_Pro_Stop Gesture_new/WIN_20180907

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.